# Walmart Sales Analysis and Insights (M5 Dataset)
**Autor:** Miguel Ángel  
**Fecha:** Septiembre 2025  

Exploratory Data Analysis (EDA) of Walmart sales data (M5 dataset). The notebook covers data cleaning, SQL-based queries, and Python visualizations to uncover insights on sales performance across stores, categories, and time periods.

The analysis focuses on:
- Data modeling and preparation  
- SQL queries for key business insights  
- Python visualizations (pandas, matplotlib)  

## 1. Importing Libraries

In [ ]:
import pandas as pd
import pyodbc as db
import matplotlib.pyplot as plt

## 2. Data Integrity Analysis

The original sales_train_validation.csv file contains daily sales data for each product and store, where each day *(d_1 … d_1913)* is represented as a separate column. To facilitate analysis, the table was reshaped from a wide format to a long format using the melt function. In the transformed table *(sales_long)*, each row now represents a unique combination of product, store, and day (d), with the corresponding number of units sold. This structure makes it easier to perform aggregations, joins with the calendar and price tables, and time-series analysis.

In [ ]:
sales_original = pd.read_csv("sales_train_validation.csv")
id_columns = ["item_id", "dept_id", "cat_id", "store_id"]
value_columns = [c for c in sales_original.columns if c.startswith("d_")]

sales_formatted = sales_original.melt(
    id_vars = id_columns,
    value_vars = value_columns,
    var_name = "d",   
    value_name = "sales"
)

sales_formatted.to_csv("sales_formatted.csv", index=False)
## Execution time: 1m 27s

The calendar dataset includes two sets of event columns *(event_name_1, event_type_1 and event_name_2, event_type_2)* because a single day may have multiple events *(e.g., Super Bowl and Cinco de Mayo on the same day)*.
In this analysis, I kept the dataset in its original form for simplicity and because it does not affect data integrity. However, as a potential improvement, these event columns could be normalized into a separate table with one event per row, making the analysis of multiple events on the same day easier and more scalable.

### Importing CSV Files

In [ ]:
calendar = pd.read_csv("calendar.csv")
sales_formatted = pd.read_csv("sales_formatted.csv")
sell_prices = pd.read_csv("sell_prices.csv")

In [ ]:
# Analyze general structure
calendar.info()
sales_formatted.info()
sell_prices.info()

In [ ]:
# Reviewing null data
calendar.isnull().sum() # -> Only nulls in events
sales_formatted.isnull().sum() # -> Not nulls
sell_prices.isnull().sum() # -> Not nulls

In [ ]:
# Reviewing duplicated data
calendar.duplicated().sum() # -> There's no duplicated data
sales_formatted.duplicated().sum() # -> There's no duplicated data
sell_prices.duplicated().sum() # -> There's no duplicated data

In [ ]:
# Validating keys and relationships
sales_formatted.set_index(["store_id", "item_id", "d"]).index.is_unique # -> There's no duplicated data that violate ingregity
sell_prices.set_index(["store_id", "item_id", "wm_yr_wk"]).index.is_unique # -> There's no duplicated data that violate ingregity

In [ ]:
# Reviewing range and logic values
(sales_formatted["sales"] < 0).sum() # -> No negative salea
(sell_prices["sell_price"] < 0).sum() # -> No negative prices

In [ ]:
# Reviewing date types
calendar["date"].min(), calendar["date"].max() # ('2011-01-29', '2016-06-19')
len(calendar) # -> 1969 days in the record
calendar.groupby("wm_yr_wk")["d"].count().value_counts().head()
# 281 commercial weeks of 7 days and 1 week of just two days (the first week of the record)

## 3. Database connection

In [ ]:
connection = db.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=MIGUESGO;DATABASE=M5_Forcasting;Trusted_Connection=yes;")

## 4. Analytics

In [ ]:
# Number of pruducts sold in all the period
query = """
    SELECT SUM(sales) AS unidades_vendidas
    FROM dbo.sales_formatted;
"""

unidades_vendidas = pd.read_sql(query, connection)
unidades_vendidas # 65,695,409 units

In [ ]:
# Total sales in all the period
query = """
    SELECT SUM(S.sales * SP.sell_price) AS total_revenue
    FROM dbo.sales_formatted AS S
    JOIN dbo.calendar AS C ON S.d = C.d
    JOIN dbo.sell_prices AS SP
        ON SP.item_id = S.item_id
        AND SP.store_id = S.store_id
        AND SP.wm_yr_wk = C.wm_yr_wk;
"""

total_revenue = pd.read_sql(query, connection)
float(total_revenue["total_revenue"]) # -> $ 187,676,570.01990828

In [ ]:
# Products sold per year in millions
query = """
    SELECT C.year,
	   SUM(S.sales) AS total_sales_by_year
    FROM dbo.sales_formatted AS S
    JOIN dbo.calendar AS C ON C.d = S.d
    GROUP BY C.year
    ORDER BY C.year
"""
total_sales_by_year = pd.read_sql(query, connection)
total_sales_by_year["total_sales_by_year"] = (
    round(total_sales_by_year["total_sales_by_year"] / 1000000, 2)
)
print(total_sales_by_year)

fig, ax = plt.subplots(figsize=(10,6))
ax.plot(total_sales_by_year["year"],
        total_sales_by_year["total_sales_by_year"],
        marker="o",
        linestyle="-",
        color="#5DA365")
for x, y in zip(total_sales_by_year["year"], total_sales_by_year["total_sales_by_year"]):
    ax.text(x, y, f"{y:,}", ha="center", va="bottom", fontsize=12)
ax.set_title("Products sold per year in millions")
ax.set_xlabel("Year")
ax.set_ylabel("Products sold")
ax.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# Total revenue per year in millions
query = """
    SELECT C.year,
        SUM(S.sales * SP.sell_price) AS total_revenue
    FROM dbo.sales_formatted AS S
    JOIN dbo.calendar AS C ON C.d = S.d
    JOIN dbo.sell_prices AS SP
        ON SP.store_id = S.store_id
        AND SP.item_id = S.item_id
        AND SP.wm_yr_wk = C.wm_yr_wk
    GROUP BY C.year;
"""
total_revenue = pd.read_sql(query, connection)
total_revenue["total_revenue"] = (
    round(total_revenue["total_revenue"] / 1000000, 2)
)
print(total_revenue)

fig, ax = plt.subplots(figsize=(10,6))
ax.plot(total_revenue["year"],
        total_revenue["total_revenue"],
        marker="o",
        linestyle="-",
        color="#59C5FF")
for x, y in zip(total_revenue["year"], total_revenue["total_revenue"]):
    ax.text(x, y, f"{y:,}", ha="center", va="bottom", fontsize=12)
ax.set_title("Total revenue per year in millions ($)")
ax.set_xlabel("Year")
ax.set_ylabel("Sales")
ax.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# -- Product that sold more units per store
query = """
    WITH total_sales_from_item AS (
        -- Total sales from every item in all stores
        SELECT store_id,
            item_id,
            SUM(sales) AS total_sales
        FROM dbo.sales_formatted
        GROUP BY store_id, item_id
    ), max_sales_from_store AS (
        -- Max sales from every store
        SELECT store_id,
        MAX(total_sales) as max_sales
        FROM total_sales_from_item
        GROUP BY store_id
    )
    SELECT TS.store_id,
        TS.item_id,
        MS.max_sales
    FROM total_sales_from_item AS TS
    JOIN max_sales_from_store AS MS
        ON TS.store_id = MS.store_id
        AND TS.total_sales = MS.max_sales
    ORDER BY store_id;
"""
total_sales_from_item = pd.read_sql(query, connection)
total_sales_from_item["store_item"] = (
    total_sales_from_item["store_id"] + " - " + total_sales_from_item["item_id"] 
)
print(total_sales_from_item[["store_item", "max_sales"]])

fig, ax = plt.subplots(figsize=(10,6))
bars = ax.barh(total_sales_from_item["store_item"],
               total_sales_from_item["max_sales"],
               color="#5DA365")
ax.bar_label(bars, padding=3, fmt="%.0f")
ax.set_title("Products that sold more units per store")
ax.set_xlabel("Units sold")
ax.set_ylabel("Store - Product")

plt.tight_layout()
plt.show()

In [ ]:
# -- Product that sold more items per department
query = """
    WITH total_sales_by_product AS (
        -- total sales by product overall
        SELECT dept_id,
            item_id,
            SUM(sales) AS total_sales
        FROM dbo.sales_formatted
        GROUP BY dept_id, item_id
    ), max_sales AS (
        SELECT dept_id,
            MAX(total_sales) AS max_sales
        FROM total_sales_by_product
        GROUP BY dept_id
    )
    SELECT TS.dept_id,
        TS.item_id,
        MS.max_sales
    FROM total_sales_by_product AS TS
    JOIN max_sales AS MS
        ON TS.dept_id = MS.dept_id
        AND TS.total_sales = MS.max_sales
    ORDER BY dept_id;
"""

total_sales_by_product = pd.read_sql(query, connection)
total_sales_by_product["dept_item"] = (
    total_sales_by_product["dept_id"] + " - " + total_sales_by_product["item_id"]
)
print(total_sales_by_product[["dept_item", "max_sales"]])

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(total_sales_by_product["dept_item"],
               total_sales_by_product["max_sales"],
               color="#59C5FF")
ax.bar_label(bars, padding=3, fmt="%.0f")
ax.set_title("Product that sold more per departament")
ax.set_xlabel("Units sold")
ax.set_ylabel("Departament - Product")

plt.tight_layout()
plt.show()

In [ ]:
# -- Product that sold more units per category
query = """
    WITH total_sales_by_product AS (
        -- total sales by product overall
        SELECT cat_id,
            item_id,
            SUM(sales) AS total_sales
        FROM dbo.sales_formatted
        GROUP BY cat_id, item_id
    ), max_sales AS (
        SELECT cat_id,
            MAX(total_sales) AS max_sales
        FROM total_sales_by_product
        GROUP BY cat_id
    )
    SELECT TS.cat_id,
        TS.item_id,
        MS.max_sales
    FROM total_sales_by_product AS TS
    JOIN max_sales AS MS
        ON TS.cat_id = MS.cat_id
        AND TS.total_sales = MS.max_sales
    ORDER BY cat_id;
"""

total_sales_by_product = pd.read_sql(query, connection)
total_sales_by_product["cat_item"] = (
    total_sales_by_product["cat_id"] + " - " + total_sales_by_product["item_id"]
)
print(total_sales_by_product[["cat_item", "max_sales"]])

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(total_sales_by_product["cat_item"],
               total_sales_by_product["max_sales"],
               color="#5DA365")
ax.bar_label(bars, padding=3, fmt="%.0f")
ax.set_title("Product that sold more per category")
ax.set_xlabel("Units sold")
ax.set_ylabel("Category - Product")

plt.tight_layout()
plt.show()

In [ ]:
# Top thre products per store
query = """
    -- TOP 3 products that sold more units by store
    WITH top_three_products AS (
        SELECT store_id,
            item_id,
            SUM(sales) AS total_sales,
            ROW_NUMBER() OVER (
                    PARTITION BY store_id
                    ORDER BY SUM(sales) DESC
            ) AS rn
        FROM dbo.sales_formatted
        GROUP BY store_id, item_id
    )
    SELECT store_id,
        item_id,
        total_sales
    FROM top_three_products
    WHERE rn <= 3
    ORDER BY store_id, rn
"""

top_three_products = pd.read_sql(query, connection)
print(top_three_products.head())

fig, ax = plt.subplots(figsize=(10, 6))
pivot = top_three_products.pivot(index="store_id",
                                 columns="item_id",
                                 values="total_sales")
pivot.plot(kind="bar", ax=ax)

ax.set_title("Top 3 products sold per store")
ax.set_xlabel("Store")
ax.set_ylabel("Units sold")
ax.legend(title="Product")
plt.tight_layout()
plt.show()

In [ ]:
# Top three products per category
query = """
    -- Total revenue by store and category
    SELECT S.store_id,
        S.cat_id,
        SUM(S.sales * SP.sell_price) AS revenue
    FROM dbo.sales_formatted AS S
    JOIN dbo.calendar AS C ON C.d = S.d
    JOIN dbo.sell_prices AS SP
        ON SP.item_id = S.item_id
        AND SP.store_id = S.store_id
        AND SP.wm_yr_wk = C.wm_yr_wk
    GROUP BY S.store_id, S.cat_id
    ORDER BY S.store_id, cat_id, revenue
"""
revenue = pd.read_sql(query, connection)
print(revenue.head())

fig, ax = plt.subplots(figsize=(10, 6))
pivot = revenue.pivot(index="store_id",
                      columns="cat_id",
                      values="revenue")
pivot.plot(kind="bar", ax=ax)

ax.set_title("Total revenue per category of each store")
ax.set_xlabel("Store")
ax.set_ylabel("Units sold")
ax.legend(title="Category")
plt.tight_layout()
plt.show()

In [ ]:
# Total sales per weekday
query = """
    SELECT C.weekday,
	   SUM(S.sales) AS sum_units
    FROM dbo.sales_formatted AS S
    JOIN dbo.calendar AS C ON C.d = S.d
    GROUP BY C.weekday
    ORDER BY sum_units
"""

weekday = pd.read_sql(query, connection)
weekday["sum_units"] = (
    round(weekday["sum_units"] / 1000000, 2)
)
print(weekday)

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(weekday["weekday"],
               weekday["sum_units"],
               color="#59C5FF")
ax.bar_label(bars, padding=3, fmt="%.0f")
ax.set_title("Total revenue per each weekday in millions")
ax.set_xlabel("Units sold - millions")
ax.set_ylabel("Weekday")

plt.tight_layout()
plt.show()

In [ ]:
# -- Week with fewer sales and week with more sales
query = """
    SELECT 
        COALESCE(C.event_name_1, C.event_name_2) AS event_name,
        SUM(S.sales) AS total_units
    FROM dbo.sales_formatted S
    JOIN dbo.calendar C ON C.d = S.d
    WHERE C.event_name_1 IS NOT NULL OR C.event_name_2 IS NOT NULL
    GROUP BY COALESCE(C.event_name_1, C.event_name_2)
    ORDER BY total_units DESC;
"""

event_sales = pd.read_sql(query, connection)
event_sales = event_sales[1:]
print(event_sales.head())

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(event_sales["event_name"],
               event_sales["total_units"],
               color="#59C5FF")
ax.bar_label(bars, padding=3, fmt="%.0f")
ax.set_title("Total sales per holiday")
ax.set_xlabel("Units sold")
ax.set_ylabel("Holiday")

plt.tight_layout()
plt.show()

In [ ]:
# -- Percentage of revenue and sales from every store
query = """
    WITH quantity_sales_per_store AS (
        SELECT S.store_id,
            SUM(S.sales) AS total_sales_per_store,
            SUM(S.sales * SP.sell_price) AS total_profit_per_store
        FROM dbo.sales_formatted AS S
        JOIN dbo.calendar AS C ON C.d = S.d
        JOIN dbo.sell_prices AS SP
            ON SP.item_id = S.item_id
            AND SP.store_id = S.store_id
            AND SP.wm_yr_wk = C.wm_yr_wk
        GROUP BY S.store_id
    ), quantity_sales_overall AS (
        SELECT SUM(S.sales) AS total_sales_overall,
            SUM(S.sales * SP.sell_price) AS total_profit_overall
        FROM dbo.sales_formatted AS S
        JOIN dbo.calendar AS C ON C.d = S.d
        JOIN dbo.sell_prices AS SP
            ON SP.item_id = S.item_id
            AND SP.store_id = S.store_id
            AND SP.wm_yr_wk = C.wm_yr_wk
    )
    SELECT QSS.store_id,
        ROUND(100.0 * QSS.total_sales_per_store / QSO.total_sales_overall, 2) AS total_sales_percentage,
        ROUND(100.0 * QSS.total_profit_per_store / QSO.total_profit_overall, 2) AS total_profit_percentage
    FROM quantity_sales_per_store AS QSS
    CROSS JOIN quantity_sales_overall AS QSO
    ORDER BY total_sales_percentage, total_profit_percentage;
"""

total_percentages = pd.read_sql(query, connection)
total_percentages

# Crear dos gráficos de pastel lado a lado
fig, axes = plt.subplots(1, 2, figsize=(14,6))

# Primer pastel: porcentaje de ventas en unidades
axes[0].pie(total_percentages["total_sales_percentage"],
            labels=total_percentages["store_id"],
            autopct="%.1f%%",
            startangle=90,
            counterclock=False)
axes[0].set_title("(%) Participation (sales)")

# Segundo pastel: porcentaje de ventas en dinero (profit)
axes[1].pie(total_percentages["total_profit_percentage"],
            labels=total_percentages["store_id"],
            autopct="%.1f%%",
            startangle=90,
            counterclock=False)
axes[1].set_title("(%) Participation (revenue)")

plt.tight_layout()
plt.show()